In [1]:
import requests

url = "https://www.allrecipes.com/recipe/12151/banana-cream-pie-i/"
r = requests.get(url)
r.content[:1000]

b'<!DOCTYPE html>\n<html id="recipeScTemplate_1-0" class="comp no-js taxlevel-5   recipeScTemplate html mntl-html" data-ab="99,99,82,99,54,99,99,99,70,72,99,64,63" data-resource-version="1.123.0" lang="en" data-lazy-offset="200" data-mantle-resource-version="3.14.335" data-allrecipes-resource-version="1.123.0" data-tracking-container="true">\t\t\t\t\n<!--\n<globe-environment environment="k8s-prod" application="allrecipes" dataCenter="us-west-1"/>\n-->\n<head class="loc head">\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t<script type="text/javascript">var Mntl = window.Mntl || {};</script>\n\t\t\t\t\t\n\t\t\t\t\t    <link rel="preconnect" href="//js-sec.indexww.com">\n    <link rel="preconnect" href="//c.amazon-adsystem.com">\n    <link rel="preconnect" href="//securepubads.g.doubleclick.net">\n    <link rel="preconnect" href="//allrecipes.groceryserver.com">\n    <link rel="preconnect" href="//products.polaris.me">\n    <link rel="preconnect" href="//calvera.allrecipes.com">\n\n\t\t\t\t\t\n\t\t\t\t\

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.content, "html.parser")

In [3]:
ingredients = soup.select("#mntl-structured-ingredients_1-0 > ul > li > p")
for ingredient in ingredients:
    print(ingredient)
    

<p><span data-ingredient-quantity="true">¾</span> <span data-ingredient-unit="true">cup</span> <span data-ingredient-name="true">white sugar</span></p>
<p><span data-ingredient-quantity="true">⅓</span> <span data-ingredient-unit="true">cup</span> <span data-ingredient-name="true">all-purpose flour</span></p>
<p><span data-ingredient-quantity="true">¼</span> <span data-ingredient-unit="true">teaspoon</span> <span data-ingredient-name="true">salt</span></p>
<p><span data-ingredient-quantity="true">2</span> <span data-ingredient-unit="true">cups</span> <span data-ingredient-name="true">milk</span></p>
<p><span data-ingredient-quantity="true">3</span> <span data-ingredient-unit="true"></span> <span data-ingredient-name="true">egg yolks, beaten</span></p>
<p><span data-ingredient-quantity="true">2</span> <span data-ingredient-unit="true">tablespoons</span> <span data-ingredient-name="true">butter</span></p>
<p><span data-ingredient-quantity="true">1 ¼</span> <span data-ingredient-unit="true

In [4]:
# from typing import List, TypedDict, Union

# class Ingredient(TypedDict):
#     quantity: str
#     unit: str
#     name: str
#     preparation: str
    
# def get_ingredients(soup: BeautifulSoup) -> List[Ingredient]:
#     ingredients = []
#     for ingredient in soup.select("#mntl-structured-ingredients_1-0 > ul > li > p"):
#         quantity = ingredient.find("span", {"data-ingredient-quantity": "true"}).text
#         unit = ingredient.find("span", {"data-ingredient-unit": "true"}).text
#         name_text = ingredient.find("span", {"data-ingredient-name": "true"}).text
#         if ", " in name_text:
#             name, preparation = name_text.split(", ", 1)
#         else:
#             name = name_text
#             preparation = ""
        
#         current_ingredient = Ingredient(
#             quantity=quantity,
#             unit=unit,
#             name=name,
#             preparation=preparation
#         )
#         ingredients.append(current_ingredient)
#     return ingredients

from ingredient import parse_ingredients

parsed_ingredients = parse_ingredients(soup)
for ingredient in parsed_ingredients:
    print(ingredient)
    

quantity: 0.75, unit: cup, name: white sugar, preparation: 
quantity: 0.3333333333333333, unit: cup, name: all-purpose flour, preparation: 
quantity: 0.25, unit: teaspoon, name: salt, preparation: 
quantity: 2.0, unit: cups, name: milk, preparation: 
quantity: 3.0, unit: , name: egg yolks, preparation: beaten
quantity: 2.0, unit: tablespoons, name: butter, preparation: 
quantity: 1.25, unit: teaspoons, name: vanilla extract, preparation: 
quantity: 1.0, unit: (9 inch), name: baked pastry shell, preparation: cooled
quantity: 4.0, unit: , name: bananas, preparation: sliced


In [5]:
from typing import List, TypedDict, Union

class Temperature(TypedDict):
    F: int
    C: int

class Action(TypedDict):
    sentence: str f"{}"
    Temperature: Union[Temperature, None]
    ingredients: List[Ingredient]
    time: int
    cooking_method: str
    tools: List[str]

class Step(TypedDict):
    actions: List[Action]

def imperative_to_normal(sentence: str) -> str:
    if sentence.endswith("."):
        return f"You {sentence[0].lower()}{sentence[1:]}"
    else:
        return f"You {sentence[0].lower()}{sentence[1:]}."
    
def split_into_sentences(text: str) -> List[str]:
    return text.strip().replace(";", ".").split(". ")

def get_steps(soup: BeautifulSoup) -> List[Step]:
    steps = []

    for step in soup.select("#mntl-sc-block_2-0 > li > p"):
        step_sentences = split_into_sentences(step.text)   
        step_actions = [Action(sentence=imperative_to_normal(sentence)) for sentence in step_sentences if sentence != ""]
        steps.append(Step(actions=step_actions))
    return steps

parsed_steps = get_steps(soup)
for step in parsed_steps:
    print(step)

{'actions': [{'sentence': 'You preheat the oven to 350 degrees F (175 degrees C).'}]}
{'actions': [{'sentence': 'You combine sugar, flour, and salt in a saucepan.'}, {'sentence': 'You gradually stir in milk.'}, {'sentence': 'You cook, stirring constantly, over medium heat until mixture boils and thickens.'}, {'sentence': 'You continue to cook and stir for 2 more minutes, then remove from the heat.'}]}
{'actions': [{'sentence': 'You place egg yolks in a medium bowl.'}, {'sentence': 'You whisk in a small amount of the hot milk mixture until smooth, then gradually whisk egg yolk mixture into the saucepan.'}, {'sentence': 'You cook over medium-low heat, stirring constantly until mixture coats the back of a spoon, about 2 more minutes.'}, {'sentence': 'You remove from heat and stir in butter and vanilla.'}]}
{'actions': [{'sentence': 'You fill pastry shell with sliced bananas.'}, {'sentence': 'You pour pudding mixture on top to cover.'}]}
{'actions': [{'sentence': 'You bake in the preheated

In [8]:
import spacy
from spacy import displacy

spacy_model = spacy.load("en_core_web_sm")

sentence = 'You preheat the oven to 350 degrees Fahrenheit (175 degrees Celsius).'

doc = spacy_model(sentence)
print([n.text for n in doc.noun_chunks])

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [8]:
import spacy
from spacy import displacy

spacy_model = spacy.load("en_core_web_lg")

def find_most_related_verb(noun_chunk: spacy.tokens.Span) -> Union[spacy.tokens.Token, None]:
    """Find the most related verb of a noun chunk (the closest ancestor that is a verb)."""
    cur_token = noun_chunk.root
    
    while cur_token.head.pos_ != "VERB":
        if cur_token.head == cur_token:
            return None
        cur_token = cur_token.head
    return cur_token.head

for step in parsed_steps:
    for action in step["actions"]:
        print(action["sentence"])
        spacy_output = spacy_model(action["sentence"])
        for n in spacy_output.noun_chunks:
            if n.text == "You":
                continue
            verb = find_most_related_verb(n)
            if verb is not None:
                print(f"{n.text} ({n.root.pos_}) -> {verb.text} ({verb.pos_})")
            else:
                print(f"{n.text} ({n.root.pos_}) -> None")
        displacy.render(spacy_output, style="dep", jupyter=True)
        

You preheat the oven to 350 degrees F (175 degrees C).
the oven (NOUN) -> preheat (VERB)
350 degrees (NOUN) -> preheat (VERB)
F (PROPN) -> preheat (VERB)
(175 degrees (NOUN) -> preheat (VERB)
C (PROPN) -> preheat (VERB)


You combine sugar, flour, and salt in a saucepan.
sugar (NOUN) -> combine (VERB)
flour (NOUN) -> combine (VERB)
salt (NOUN) -> combine (VERB)
a saucepan (NOUN) -> combine (VERB)


You gradually stir in milk.
milk (NOUN) -> stir (VERB)


You cook, stirring constantly, over medium heat until mixture boils and thickens.
medium heat (NOUN) -> stirring (VERB)
mixture (NOUN) -> boils (VERB)


You continue to cook and stir for 2 more minutes, then remove from the heat.
2 more minutes (NOUN) -> stir (VERB)
the heat (NOUN) -> remove (VERB)


...

You remove from heat and stir in butter and vanilla.
heat (NOUN) -> remove (VERB)
butter (NOUN) -> stir (VERB)
vanilla (NOUN) -> stir (VERB)


You fill pastry shell with sliced bananas.
pastry shell (NOUN) -> fill (VERB)
sliced bananas (NOUN) -> fill (VERB)


You pour pudding mixture on top to cover.
pudding mixture (NOUN) -> pour (VERB)
top (NOUN) -> pour (VERB)


You bake in the preheated oven until filling sets, 12 to 15 minutes.
the preheated oven (NOUN) -> bake (VERB)
sets (NOUN) -> filling (VERB)


You let pie cool completely on a wire rack, then chill pie in the refrigerator for at least 1 hour before serving.
pie (NOUN) -> cool (VERB)
a wire rack (NOUN) -> cool (VERB)
pie (NOUN) -> chill (VERB)
the refrigerator (NOUN) -> chill (VERB)
